In [1]:
!pip install soynlp
!pip show soynlp
!pip install gensim
!pip show gensim

     |████████████████████████████████| 416 kB 5.1 MB/s 
Name: soynlp
Version: 0.0.493
Summary: Unsupervised Korean Natural Language Processing Toolkits
Home-page: https://github.com/lovit/soynlp
Author: Lovit
Author-email: soy.lovit@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: psutil, scikit-learn, scipy, numpy
Required-by: 
Name: gensim
Version: 3.6.0
Summary: Python framework for fast Vector Space Modelling
Home-page: http://radimrehurek.com/gensim
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPLv2.1
Location: /usr/local/lib/python3.7/dist-packages
Requires: six, scipy, numpy, smart-open
Required-by: 


In [2]:
from soynlp.tokenizer import RegexTokenizer
import pandas as pd
import re
tokenizer = RegexTokenizer()
tokenizer

/usr/local/lib/python3.7/dist-packages/soynlp/tokenizer/_tokenizer.py:19: FutureWarning: Possible nested set at position 13
  ('english & latin', re.compile(u"[a-zA-ZÀ-ÿ]+[[`']?s]*|[a-zA-ZÀ-ÿ]+", re.UNICODE))


In [3]:
text=pd.read_csv("/content/all.csv",encoding = 'utf-8-sig')

In [4]:
text["total"] = text["실거래품명"] + " "+ text["물품설명"] + " " + text["결정사유"]

In [ ]:
text["total"].to_csv('/content/total2.txt',index = False)

In [5]:
#실거래품명, 물품설명, 결정사유
sen = text["total"]

In [6]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [7]:
for i in range(len(sen)):
  sample_content = preprocessing(str(sen[i]))
  sen[i] = sample_content

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
from soynlp.tokenizer import RegexTokenizer

tokenizer = RegexTokenizer()
tokenizer

In [9]:
# 전처리 이후의 샘플 텍스트로 토큰화
tokened_content = tokenizer.tokenize(sen[0])
tokened_content[:10]

['살아있는', '자라', '본', '건은', '자라과에', '속하는', '파충류의', '일종으로', '생후', '개월크기']

In [ ]:
%time tokens = sen.apply(tokenizer.tokenize)
tokens[:]

In [ ]:
%pip install konlpy

In [ ]:
from konlpy.tag import Okt

In [ ]:
def read_data(filename, encoding='utf-8'):#->[]
    with open(filename, 'r',encoding=encoding) as f:
        data= [line.split(' ') for line in f.read().splitlines()]
    return data

In [ ]:
stopwords = read_data('/content/stopwords.txt')

In [27]:
okt = Okt()
tokenized_data = []
total_lines =len(sen)
cnt =0
for sentence in sen:
  temp_X = okt.morphs(sentence, stem = True)
  temp_X = [word for word in temp_X if not word in stopwords]
  tokenized_data.append(temp_X)
  cnt += 1
  if(cnt %1000) == 0:
    print(round(cnt/total_lines * 100, 3),'%')

1.261 %


KeyboardInterrupt: ignored

In [ ]:
from gensim.models import FastText
model = FastText(tokenized_data, size=150, window=10, min_count=5, workers=4, sg=1)

In [ ]:
def create_n_save_embedding_model(data): # -> void
    embedding_dim =150
    # size : 벡터의 차원 , 
    # window: 컨텍스트 윈도우의 크기, 문장 내에서 현재 단어와 예측 단어 사이의 최대 거리
    # min_count 단어 최소 빈도, 총 빈도가 이보다 작은 모든 단어를 무시한다.
    # workers: :학습을 위한 프로세스 수
    # sg : 0 = CBOW, 1= skip-gram
    model = FastText(sentences = data, size=embedding_dim, window=10, min_count=5, workers=4, sg=1)
    
    # 모델 저장
    model.save('embedding.model')

    words = list(model.wv.vocab)
    with open('/content/embedding_window10_mincnt5_skipgram.txt', 'w') as f:
        for word in words:
            emb_value = model.wv[word].tolist() # 현재 단어의 임베딩 값을 가져온다.
            print('value: ',emb_value)

            emb_value.insert(0,word)            # 시작 부분에 해당 단어를 넣는다.
            print('value_after: ',emb_value)

            for item in emb_value:              # 단어 이름부터 시작하여 각 벡터의 값을 저장한다.
                f.write('%s ' % item)
            f.write('\n')

In [ ]:
create_n_save_embedding_model(tokenized_data)

In [ ]:
model.wv.vectors.shape

In [ ]:
# 단어 사전에서 상위 10개만 보기
vocab = model.wv.vocab
sorted(vocab, key=vocab.get, reverse=True)[:100]

In [ ]:
# 가장 적게 등장하는 단어
min(vocab, key=vocab.get)

In [ ]:
model.wv.most_similar('샤프심')

In [ ]:
print('--- 유사단어 출력 ---')
similarWords = model.wv.most_similar(positive=['샤프심', '샤프', '심'], topn=3)
print(similarWords) # [('흐뭇', 0.8801068067550659), ('마음', 0.8738192319869995), ('해맑', 0.8424177169799805)]
